In [2]:
import os
%load_ext tensorboard

In [3]:
import random
import numpy as np
import tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
# utils
from distutils.version import LooseVersion
from tqdm import tqdm_notebook
from datetime import datetime
from time import time

import warnings
import pickle
import gc
import sys
from json import dumps
import itertools
import re

# Data
import pandas as pd
import spacy

# Viz
import matplotlib.pyplot as plt

# Machine Learning
import tensorflow.keras.backend as K
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Bidirectional, LSTM, Flatten, Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks.callbacks import EarlyStopping
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHead, MultiHeadAttention

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# NLP Models
from gensim.models import Word2Vec, KeyedVectors
w2v_models_path = 'D:/Mestrado/Dissertação/07 .Dissertação Final/02. Experimentos/02. Word Embbedings/'

Using TensorFlow backend.


In [5]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
print('tf version: ' + tf.__version__)

tf version: 2.1.0-rc2


In [7]:
#Variables
current_exp = 'Clareza-Balanced-Multiclass-COH-METRIX'
if 'Binary' in current_exp:
    binary = True
else:
    binary = False
    
base_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/03. Datasets/' + current_exp
save_path = 'output'

sentence = 'resp-text'
label = 'Clareza'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train[label].value_counts())
print(y_test[label].value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       'sentence_length_min', 'sentence_length_standard_deviation',
       'short_sentence_ratio', 'std_noun_phrase', 'verb_diversity',
       'verbs_max', 'verbs_min', 'verbs_standard_deviation',
       'long_sentence_ratio', 'tempo_resposta'],
      dtype='object', length=310)
(4832, 310)
0    1628
1    1623
2    1581
Name: Clareza, dtype: int64
2    720
1    678
0    673
Name: Clareza, dtype: int64


In [8]:
# Keep only text columns
X_train.drop(columns=X_train.columns[3:], inplace=True)
X_test.drop(columns=X_train.columns[3:], inplace=True)

In [9]:
###################################################
X_train['sentence'] = X_train[sentence]
X_test['sentence'] = X_test[sentence]

y_train['label'] = y_train[label]
y_test['label'] = y_test[label]

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(X_train.shape[0]))

# Report the classes balance.
print('Classes distribuition: \n')
print(y_train[label].value_counts())

# Display 10 random rows from the data.
X_train.sample(10)

Number of training sentences: 4,832

Classes distribuition: 

0    1628
1    1623
2    1581
Name: Clareza, dtype: int64


,pid,req-text,resp-text,sentence
746,424138,Gostaria de solicitar a autorização para consu...,"Prezado Senhor , Em atenção ao pedido de infor...","Prezado Senhor , Em atenção ao pedido de infor..."
1550,299165,"Venho, por meio desta, solicitar acesso à: - P...",Prezado A publicação da comissão de seleção es...,Prezado A publicação da comissão de seleção es...
4530,341237,"O poque que nao recebi o meu PIS 2015, ja que ...",Prezado a Senhor a 1 . Em atenção ao pedido d...,Prezado a Senhor a 1 . Em atenção ao pedido d...
3754,303947,"Prezados Senhores, Solicito a cópia colorida d...","Prezado Sr. Edilson, A UFSJ não dispõe de tal ...","Prezado Sr. Edilson, A UFSJ não dispõe de tal ..."
3853,433542,"Prezados bom dia. Favor, gostaria de saber da ...","Prezado Senhor , Informamos que a Receita Fede...","Prezado Senhor , Informamos que a Receita Fede..."
2651,440861,"Prezados Senhores da ANAC, Solicito, nos termo...","Prezado Senhor , Em referência à demanda apres...","Prezado Senhor , Em referência à demanda apres..."
1773,523200,"Bom dia. Gostaria de pedir, por gentileza, se ...","Prezada senhora , este Ministério não possui ...","Prezada senhora , este Ministério não possui ..."
4731,451684,Gostaria de saber como consigo dados qualitati...,"Prezado a Senhor a , Em atendimento ao pedido ...","Prezado a Senhor a , Em atendimento ao pedido ..."
4256,353993,Solicito informações acerca do Plano de Ação r...,"Prezada Senhora Patrícia , Enviamos a solicita...","Prezada Senhora Patrícia , Enviamos a solicita..."
4051,306410,Sou graduanda de Oceanografia na Universidade ...,"Prezado Senhor, Temos a esclarecer que recebem...","Prezado Senhor, Temos a esclarecer que recebem..."


In [10]:
# Looking lengths
lengths = [X_train.sentence.apply(lambda x: len(x.split(' ')))]
perc =[.25, .50, .75, .80, .85, .90, .91, .92, .93, .94, .95, .96, .97, .98, .99] 
lengths[0].describe(percentiles = perc)

count    4832.000000
mean      172.953022
std       157.267000
min         2.000000
25%        60.000000
50%       129.000000
75%       239.000000
80%       270.000000
85%       302.000000
90%       363.000000
91%       378.000000
92%       396.000000
93%       415.000000
94%       443.000000
95%       474.350000
96%       509.000000
97%       545.000000
98%       640.760000
99%       778.690000
max      1155.000000
Name: sentence, dtype: float64

# w2v Model for Embedding Layer

In [11]:
#w2v_cbow_esic_model=KeyedVectors.load(os.path.join(w2v_models_path,'word2vec_sg_hs_DetalhamentoSolicitacao_all_sentences_128.model'))
w2v_cbow_nilc_model=KeyedVectors.load_word2vec_format(os.path.join(w2v_models_path,'cbow_s300.txt'))

In [12]:
pretrained_weights = w2v_cbow_nilc_model.wv.syn0
print(pretrained_weights.shape)
max_num_words = pretrained_weights.shape[0]
embed_size = pretrained_weights.shape[1]

(929606, 300)


C:\Users\arthu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
C:\Users\arthu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


# Data Prep

In [13]:
max_length=256

# Define tokenizer and fit train data
t = Tokenizer(num_words=max_num_words)
t.fit_on_texts(X_train['sentence'].append(X_test['sentence']))
word_index = t.word_index
vocab_size = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))
    
def get_seqs(text):    
    sequences = t.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

Found 29662 unique tokens.


In [14]:
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    if binary:
        return y_train_enc, y_test_enc
    else:
        return pd.get_dummies(y_train_enc), pd.get_dummies(y_test_enc)

In [15]:
# X and Y
label_train, label_test = prepare_targets(y_train.label.values, y_test.label.values)
num_labels = len(set(label_train))
input_train = get_seqs(X_train.sentence)
input_test = get_seqs(X_test.sentence)

# Modeling

In [16]:
embedding_matrix = np.zeros((vocab_size, embed_size))
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v_cbow_nilc_model.wv.__getitem__(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    # Words not in vocab -> Frequency less than 5 word
    except KeyError as e:
        print(e)

C:\Users\arthu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


"word 'à' not in vocabulary"
"word '12' not in vocabulary"
"word '1' not in vocabulary"
"word '7' not in vocabulary"
"word '10' not in vocabulary"
"word '2' not in vocabulary"
"word '2012' not in vocabulary"
"word '724' not in vocabulary"
"word 'às' not in vocabulary"
"word '527' not in vocabulary"
"word '2011' not in vocabulary"
"word '3' not in vocabulary"
"word '2016' not in vocabulary"
"word '2015' not in vocabulary"
"word '4' not in vocabulary"
"word '0800' not in vocabulary"
"word '21' not in vocabulary"
"word '2017' not in vocabulary"
"word '5' not in vocabulary"
"word '2014' not in vocabulary"
"word '18' not in vocabulary"
"word '11' not in vocabulary"
"word '15' not in vocabulary"
"word '–' not in vocabulary"
"word 'acessoainformacao' not in vocabulary"
"word '13' not in vocabulary"
"word '1ª' not in vocabulary"
"word '08' not in vocabulary"
"word '07' not in vocabulary"
"word '61' not in vocabulary"
"word '16' not in vocabulary"
"word '01' not in vocabulary"
"word '24' not in

"word '305' not in vocabulary"
"word '197' not in vocabulary"
"word '07h' not in vocabulary"
"word '503' not in vocabulary"
"word '6333' not in vocabulary"
"word '3319' not in vocabulary"
"word 'sgtes' not in vocabulary"
"word '830' not in vocabulary"
"word 'agrofit' not in vocabulary"
"word 'narelação' not in vocabulary"
"word '142' not in vocabulary"
"word 'naferramenta' not in vocabulary"
"word '169' not in vocabulary"
"word 'sigaa' not in vocabulary"
"word 'anapoderá' not in vocabulary"
"word '124' not in vocabulary"
"word '534' not in vocabulary"
"word '256' not in vocabulary"
"word '134' not in vocabulary"
"word 'a3o' not in vocabulary"
"word 'nabusca' not in vocabulary"
"word '7283' not in vocabulary"
"word '8004' not in vocabulary"
"word 'naconta' not in vocabulary"
"word '2179' not in vocabulary"
"word 'destinaà' not in vocabulary"
"word '230' not in vocabulary"
"word 'cget' not in vocabulary"
"word '70070' not in vocabulary"
"word 'geout' not in vocabulary"
"word '3314' not i

"word '419' not in vocabulary"
"word '067' not in vocabulary"
"word '850' not in vocabulary"
"word '231' not in vocabulary"
"word '849' not in vocabulary"
"word 'cgesp' not in vocabulary"
"word '452' not in vocabulary"
"word 'formulariomensagem' not in vocabulary"
"word 'eletrobrasroraima' not in vocabulary"
"word 'cgue' not in vocabulary"
"word '1795' not in vocabulary"
"word 'naatenção' not in vocabulary"
"word '559' not in vocabulary"
"word 'digea' not in vocabulary"
"word '610' not in vocabulary"
"word 'cgpep' not in vocabulary"
"word 'aquiculturampa' not in vocabulary"
"word '6250' not in vocabulary"
"word '992' not in vocabulary"
"word '934' not in vocabulary"
"word '028' not in vocabulary"
"word 'portosdobrasil' not in vocabulary"
"word '25351' not in vocabulary"
"word 'gcpaf' not in vocabulary"
"word 'naocasião' not in vocabulary"
"word 'coeng' not in vocabulary"
"word 'acoesadministrativas' not in vocabulary"
"word 'cdoc' not in vocabulary"
"word 'protocolo2' not in vocabulary

"word 'micrrural' not in vocabulary"
"word 'cgde' not in vocabulary"
"word '000385' not in vocabulary"
"word '3360' not in vocabulary"
"word '5108' not in vocabulary"
"word '810' not in vocabulary"
"word '3114' not in vocabulary"
"word '1873' not in vocabulary"
"word '432' not in vocabulary"
"word 'cctiv' not in vocabulary"
"word 'incido' not in vocabulary"
"word 'ambiental\xa0' not in vocabulary"
"word 'inheritredirect' not in vocabulary"
"word '33836' not in vocabulary"
"word '8240' not in vocabulary"
"word '001085' not in vocabulary"
"word 'itemid' not in vocabulary"
"word '719' not in vocabulary"
"word 'pesquisajurisprudencia' not in vocabulary"
"word '2520desc' not in vocabulary"
"word 'naentrada' not in vocabulary"
"word 'irhc' not in vocabulary"
"word 'rhcnet' not in vocabulary"
"word '000364' not in vocabulary"
"word '23483' not in vocabulary"
"word '0757' not in vocabulary"
"word 'disciplinacomo' not in vocabulary"
"word 'disciplinaobrigatória' not in vocabulary"
"word 'naobte

...

"word '23480005906201717' not in vocabulary"
"word 'argentinaquanto' not in vocabulary"
"word '71200000576201715' not in vocabulary"
"word 'decef' not in vocabulary"
"word '15h14' not in vocabulary"
"word '0171' not in vocabulary"
"word 'cdfmm' not in vocabulary"
"word '53850000572201709' not in vocabulary"
"word 'orcamentos' not in vocabulary"
"word '1123' not in vocabulary"
"word '23480008413201739' not in vocabulary"
"word 'naclt' not in vocabulary"
"word '1903' not in vocabulary"
"word 'sistn' not in vocabulary"
"word '99944000043201759' not in vocabulary"
"word 'navalec' not in vocabulary"
"word 'thayzi' not in vocabulary"
"word '0504' not in vocabulary"
"word '1929' not in vocabulary"
"word '1621' not in vocabulary"
"word 'strutural' not in vocabulary"
"word 'torgone' not in vocabulary"
"word 'gepro' not in vocabulary"
"word 'kaian' not in vocabulary"
"word '5568' not in vocabulary"
"word 'brunalusa' not in vocabulary"
"word 'gerênciageral' not in vocabulary"
"word '000817' not i

"word '3529' not in vocabulary"
"word '2778' not in vocabulary"
"word '5263' not in vocabulary"
"word 'latinalogistica' not in vocabulary"
"word '000050' not in vocabulary"
"word '99920000547201666' not in vocabulary"
"word '2098' not in vocabulary"
"word '000231' not in vocabulary"
"word '6901' not in vocabulary"
"word '7574' not in vocabulary"
"word '02916' not in vocabulary"
"word '9210' not in vocabulary"
"word '25820000015201611' not in vocabulary"
"word '1656295' not in vocabulary"
"word 'n°54800001490201560' not in vocabulary"
"word 'gebio' not in vocabulary"
"word 'informaçãon°12' not in vocabulary"
"word '000520' not in vocabulary"
"word 'para0800' not in vocabulary"
"word 'psnc' not in vocabulary"
"word 'petrolinae' not in vocabulary"
"word '99903000520201762' not in vocabulary"
"word '001225' not in vocabulary"
"word '23480015570201539' not in vocabulary"
"word '23480015230201516' not in vocabulary"
"word 'indígenadsei' not in vocabulary"
"word '03100200670201589' not in voc

"word 'condicaodevida' not in vocabulary"
"word 'indicadoresminimos' not in vocabulary"
"word 'sinteseindicsociais2012' not in vocabulary"
"word '349' not in vocabulary"
"word 'scon' not in vocabulary"
"word 'cgsl' not in vocabulary"
"word '017240' not in vocabulary"
"word '005028' not in vocabulary"
"word '1173' not in vocabulary"
"word 'balbinaresolução' not in vocabulary"
"word '4244' not in vocabulary"
"word 'naaviação' not in vocabulary"
"word 'sorgatto' not in vocabulary"
"word 'naufgd' not in vocabulary"
"word 'alanarizzo' not in vocabulary"
"word 'anival' not in vocabulary"
"word '2467' not in vocabulary"
"word '48700001714201640' not in vocabulary"
"word 'ibiapinapara' not in vocabulary"
"word 'ibiapinaendereço' not in vocabulary"
"word '3653' not in vocabulary"
"word '008170' not in vocabulary"
"word '000689' not in vocabulary"
"word '004800' not in vocabulary"
"word '03950000183201779' not in vocabulary"
"word 'nasinopse' not in vocabulary"
"word '832366' not in vocabulary"


"word 'josemberto' not in vocabulary"
"word 'postiglioni' not in vocabulary"
"word 'cristinadomingues' not in vocabulary"
"word '20112' not in vocabulary"
"word '03950001260201716' not in vocabulary"
"word 'gerhum' not in vocabulary"
"word '31327353' not in vocabulary"
"word '23480011857201590' not in vocabulary"
"word 'beneficiossociais' not in vocabulary"
"word 'kedson' not in vocabulary"
"word '6105' not in vocabulary"
"word 'shellard' not in vocabulary"
"word 'cosep' not in vocabulary"
"word 'napauta' not in vocabulary"
"word 'cgvb' not in vocabulary"
"word 'nainspeção' not in vocabulary"
"word '1283' not in vocabulary"
"word 'nasfa' not in vocabulary"
"word 'phanardzis' not in vocabulary"
"word '001438' not in vocabulary"
"word '3676' not in vocabulary"
"word '99903000173201697' not in vocabulary"
"word 'inhamã' not in vocabulary"
"word 'institucionalgeral' not in vocabulary"
"word '16853007056201589' not in vocabulary"
"word '25820002897201659' not in vocabulary"
"word 'item2' no

"word '938' not in vocabulary"
"word 'l12651' not in vocabulary"
"word 'l6938' not in vocabulary"
"word 'res97' not in vocabulary"
"word 'res23797' not in vocabulary"
"word 'srhu' not in vocabulary"
"word '14001' not in vocabulary"
"word 'gefor' not in vocabulary"
"word 'catalogonct' not in vocabulary"
"word 'consultapublicaunidadeensino' not in vocabulary"
"word '18600000357201660' not in vocabulary"
"word 'dividadlsp' not in vocabulary"
"word 'dividadlsp08' not in vocabulary"
"word 'cesef' not in vocabulary"
"word '6377' not in vocabulary"
"word 'samôr' not in vocabulary"
"word 'trivale' not in vocabulary"
"word '99901001216201591' not in vocabulary"
"word 'aironserv' not in vocabulary"
"word '013279' not in vocabulary"
"word '006865' not in vocabulary"
"word '23480010349201675' not in vocabulary"
"word '7555' not in vocabulary"
"word 'grcsu' not in vocabulary"
"word '99901000578201564' not in vocabulary"
"word '2637' not in vocabulary"
"word 'whisner' not in vocabulary"


In [17]:
# Define model
def train_model(input_train, input_test, label_train, label_test,
                lstm_size=128, dropout=0.2, rec_dropout=0.2, lr=0.005, epochs=50, att_heads=4, max_length=128, 
                vocab_size=None, embed_size=None, emb_trainable=False, batch=128, early_stopping=5,
                save_dir="D:/resultados/checkpoins_solicitacao_keras_mh_att/", best_predefined_f1=0.390):

    # Time now
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Log
    log_dir = save_dir + now
    
    # Model Saver    
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    log_file = open(os.path.join(log_dir,"log.txt"), mode="a")
    
    # Save Params
    params = {
        'lstm_size': lstm_size,
        'dropout': dropout,
        'rec_dropout': rec_dropout,
        'lr': lr,
        'epochs': epochs,
        'att_heads': att_heads,
        'max_length': max_length,
        'vocab_size': vocab_size,
        'embed_size': embed_size,
        'emb_trainable': emb_trainable,
        'batch': batch,
        'early_stopping': early_stopping,
        'log_dir': log_dir,
        'best_predefined_f1': best_predefined_f1}
    
    # Saving Parameters
    with open(os.path.join(log_dir, 'params.txt'),'a') as f:
        f.write('\n\n' + ('#'*60))
        f.write('\nParameters:\n')
        f.write('now: ' + str(now))
        f.write('\n' + dumps(params) + '\n')
    
    # input
    inp = Input(shape=(max_length, ))

    # Embedding layer - https://keras.io/layers/embeddings/
    embedding_layer = Embedding(vocab_size,
                                embed_size,                                
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=emb_trainable,
                                name='Embedding')(inp)

    # Bidirectional Layer
    bilstm_layer = Bidirectional(LSTM(
                        units=lstm_size,
                        return_sequences=True,
                        dropout=dropout,
                        recurrent_dropout=rec_dropout,
                        name='LSTM'))(embedding_layer)    

    # MultiHead-Attention Layer
    #https://pypi.org/project/keras-multi-head/
    multiHead_att_layer = MultiHeadAttention(head_num=att_heads, name='Multi-Head-Attention')(bilstm_layer)

    dropout_intermed_layer = Dropout(0.5)(multiHead_att_layer)

    # # Flatten
    flatten_layer = Flatten(name='Flatten')(dropout_intermed_layer)

    #dense_intermed_layer = Dense(128, activation='relu')(flatten_layer)
    #dropout_intermed_2_layer = Dropout(dropout)(dense_intermed_layer)

    # # # # Dense Layer
    if binary:
        dense_layer = Dense(1, activation='sigmoid')(flatten_layer)    
    else:
        dense_layer = Dense(num_labels, activation='softmax')(flatten_layer)    
    
    model = Model(inputs=inp, outputs=dense_layer)
    # model.summary()
    
    # Compile
    model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy', precision_m, recall_m, f1_m])
    
    # callbacks
    es_callback = EarlyStopping(monitor='val_loss', patience=early_stopping, verbose=1, mode='min')
    
    # Fitting Model
    model.fit(input_train,
              label_train,
              epochs=epochs,
              batch_size=batch,
              validation_data=(input_test, label_test),
              verbose=0,
              callbacks=[es_callback])
    
    # PLOT LOSS
    plt.title('Loss')
    plt.plot(model.history.history['loss'], label='train')
    plt.plot(model.history.history['val_loss'], label='test')
    plt.legend()
    #plt.show();
    plt.savefig(os.path.join(log_dir,'Loss.png'))
    plt.close()
    
    # Classification
    y_pred = model.predict(input_test, batch_size=batch, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    
    if not binary:
        label_test = np.argmax(label_test.values, axis=1)
    
    # Metrics
    f1 = f1_score(label_test, y_pred_bool, average='weighted')
    print(f"Best Test F1-Score: {f1:.3f}")
    
    print("#"*60 + '\n', file=log_file)
    print(classification_report(label_test, y_pred_bool), file=log_file)
    print("#"*60+ '\n', file=log_file)
    
    # Flush log file
    log_file.flush()
    log_file.close()
    
    # Save final result
    with open(os.path.join(log_dir[:-16], 'output.txt'),'a') as f:
        f.write('\n\n')
        f.write(log_dir)
        f.write('\n')
        f.write(f"Best Test F1-Score: {f1:.3f}")
        
    # save model and architecture to single file
    if f1 > best_predefined_f1:
        model.save(os.path.join(log_dir, "model.h5"))
        print("Saved model to disk")

In [18]:
# Model Params
lstm_size_list = [256, 512]
dropout_list = [0.1, 0.25, 0.5]
rec_dropout_list = [0.1, 0.25, 0.5]
lr_list = [1e-3, 5e-4, 1e-4, 5e-5, 5e-6]

all_params = [lstm_size_list] + [dropout_list] + [rec_dropout_list] + [lr_list]

for each in itertools.product(*all_params):    
    lstm_size, dropout, rec_dropout, lr = each
    
    # Params
    print('lstm_size: ' + str(lstm_size))
    print('\tdropout: ' + str(dropout))
    print('\trec_dropout: ' + str(rec_dropout))
    print('\tlr: ' + str(lr))
    
    # train
    train_model(input_train, input_test, label_train, label_test, lstm_size, dropout, rec_dropout, lr, epochs=50,
                att_heads=4, max_length=max_length, vocab_size=vocab_size, embed_size=embed_size, emb_trainable=False,
                batch=96, early_stopping=5,
                save_dir="D:/Outputs_Mestrado/resultados_Clareza/checkpoins_resposta_keras_mh_att/",
                best_predefined_f1=0.4)

lstm_size: 256
	dropout: 0.1
	rec_dropout: 0.1
	lr: 0.001
Epoch 00011: early stopping
2071/2071 [==============================] - 4s 2ms/step
Best Test F1-Score: 0.391
lstm_size: 256
	dropout: 0.1
	rec_dropout: 0.1
	lr: 0.0005
Epoch 00009: early stopping
2071/2071 [==============================] - 4s 2ms/step
Best Test F1-Score: 0.398
lstm_size: 256
	dropout: 0.1
	rec_dropout: 0.1
	lr: 0.0001
Epoch 00009: early stopping
2071/2071 [==============================] - 4s 2ms/step
Best Test F1-Score: 0.397
lstm_size: 256
	dropout: 0.1
	rec_dropout: 0.1
	lr: 5e-05
Epoch 00013: early stopping
2071/2071 [==============================] - 5s 2ms/step
Best Test F1-Score: 0.393
lstm_size: 256
	dropout: 0.1
	rec_dropout: 0.1
	lr: 5e-06
Epoch 00034: early stopping
2071/2071 [==============================] - 4s 2ms/step
Best Test F1-Score: 0.388
lstm_size: 256
	dropout: 0.1
	rec_dropout: 0.25
	lr: 0.001
Epoch 00011: early stopping
2071/2071 [==============================] - 4s 2ms/step
Best Test

ResourceExhaustedError:  OOM when allocating tensor with shape[131072,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradients_12/dense_13/MatMul_grad/MatMul_1 (defined at C:\Users\arthu\Anaconda3\envs\keras_env\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_205913]

Function call stack:
keras_scratch_graph
